In [ ]:
from pyspark.sql import SparkSession

# Detener sesión anterior
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "20g") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

In [ ]:
ruta_df_principal = f"data/resultados/dataset_final.parquet"
ruta_df_reglas= f"data/resultados/dataset_reglas.parquet"

In [ ]:
df_dataset = spark.read.parquet(ruta_df_principal)
df_reglas = spark.read.parquet(ruta_df_reglas)

In [ ]:
df_dataset.printSchema()

In [ ]:
df_reglas.printSchema()

In [ ]:
from pyspark.sql.functions import col, explode, regexp_extract, split

# Extraer los elementos individuales de la columna "items"
df_items = df_reglas.withColumn("item", explode(split(col("items"), ",")))

# Filtrar solo los diagnósticos (Patrón: Letra + 2 números)
df_diagnosticos = df_items.withColumn("diagnostico", regexp_extract(col("item"), r"^[A-Z][0-9]{2}", 0)) \
                          .filter(col("diagnostico") != "")

# Mostrar los diagnósticos extraídos
df_diagnosticos.select("diagnostico").distinct().show(10, truncate=False)

In [ ]:
# Contar el número total de diagnósticos únicos
num_diagnosticos = df_diagnosticos.select("diagnostico").distinct().count()

print(f"Total de diagnósticos únicos en las reglas: {num_diagnosticos}")

Se ha llevado a cabo un proceso de extracción y filtrado de diagnósticos a partir del conjunto de reglas de asociación generadas previamente. Dado que el conjunto de datos original contiene una gran diversidad de diagnósticos, muchos de ellos con una granularidad excesiva o con baja representación, se han seleccionado únicamente aquellos que han sido identificados dentro de las reglas de asociación. Para ello, se han analizado los ítems presentes en los antecedentes y consecuentes de las reglas, extrayendo aquellos que cumplen con el patrón de los códigos de diagnóstico (una letra seguida de dos números). Como resultado, se obtiene un total de 1.346 diagnósticos únicos, los cuales serán la base para la construcción del árbol de decisión.

Esta selección no solo optimiza el rendimiento computacional al reducir significativamente el volumen de datos procesados, sino que también fortalece la robustez del modelo. Al enfocarnos en diagnósticos que ya han demostrado relaciones significativas dentro de las reglas de asociación, aseguramos que el árbol de decisión se construya sobre patrones clínicamente relevantes. No obstante, se es conciente de que esta estrategia podría excluir ciertos diagnósticos menos frecuentes pero potencialmente importantes, por lo que en futuros estudios sería recomendable evaluar el impacto de esta decisión y considerar estrategias que permitan incluir un mayor espectro de diagnósticos sin comprometer la escalabilidad del modelo.

In [ ]:
# Contar total de registros en df_dataset
total_registros = df_dataset.count()

# Filtrar registros cuyo dominio_icd NO esté en los diagnósticos seleccionados
registros_excluidos = df_dataset.filter(~col("dominio_icd").isin([row["diagnostico"] for row in df_diagnosticos.select("diagnostico").distinct().collect()])).count()

print(f"Total de registros en el dataset original: {total_registros}")
print(f"Registros que serían eliminados: {registros_excluidos}")
print(f"Porcentaje de registros eliminados: {(registros_excluidos / total_registros) * 100:.2f}%")

In [ ]:
# Filtrar solo los registros que contienen diagnósticos dentro del conjunto seleccionado
df_dataset_filtrado = df_dataset.filter(col("dominio_icd").isin([row["diagnostico"] for row in df_diagnosticos.select("diagnostico").distinct().collect()]))

# Verificar cuántos registros hay después del filtrado
registros_finales = df_dataset_filtrado.count()

print(f"Total de registros después del filtrado: {registros_finales}")

In [ ]:
from pyspark.sql.functions import col, sum

# Contar valores nulos por columna
valores_nulos = df_dataset_filtrado.select([
    (sum(col(c).isNull().cast("int")) / df_dataset_filtrado.count()).alias(c)
    for c in df_dataset_filtrado.columns
])


In [ ]:
valores_nulos.show()

In [ ]:
from pyspark.sql.functions import when

# Reemplazar valores nulos en 'estado_civil' y 'tipo_seguro' por "Desconocido"
df_dataset_filtrado = df_dataset_filtrado.withColumn(
    "estado_civil", when(col("estado_civil").isNull(), "Desconocido").otherwise(col("estado_civil"))
).withColumn(
    "tipo_seguro", when(col("tipo_seguro").isNull(), "Desconocido").otherwise(col("tipo_seguro"))
)

# Verificar que no haya valores nulos
df_dataset_filtrado.select("estado_civil", "tipo_seguro").summary("count").show()

In [ ]:
# Contar el número total de pruebas únicas en el dataset
total_pruebas_unicas = df_dataset_filtrado.select("id_prueba").distinct().count()
print(f"Total de pruebas únicas en el dataset: {total_pruebas_unicas}")

In [ ]:
from pyspark.sql.functions import regexp_extract

# Extraer items de las reglas
df_items = df_reglas.withColumn("item", explode(split(col("items"), ",")))

# Filtrar solo los ID de prueba (Patrón exactamente de 5 dígitos)
df_pruebas_reglas = df_items.withColumn("prueba", regexp_extract(col("item"), r"^\d{5}$", 0)) \
                            .filter(col("prueba") != "") \
                            .select("prueba").distinct()

# Contar el número de pruebas en las reglas de asociación
pruebas_en_reglas_count = df_pruebas_reglas.count()
print(f"Total de pruebas representadas en las reglas de asociación: {pruebas_en_reglas_count}")

In [ ]:
df_dataset.show()

# Creación del dataset para el arbol de decision

In [ ]:
from pyspark.sql.functions import collect_set, when, col, array_contains

# Obtener el listado único de pruebas
pruebas_unicas = df_dataset_filtrado.select("id_prueba").distinct().rdd.flatMap(lambda x: x).collect()

# Agrupar por `id_ingreso` y almacenar las pruebas realizadas en una lista
df_pruebas = df_dataset_filtrado.groupBy("id_ingreso").agg(
    collect_set("id_prueba").alias("pruebas_realizadas")
)

# Convertir la lista de pruebas en variables dicotómicas (One-Hot Encoding) usando `array_contains()`
for p in pruebas_unicas:
    df_pruebas = df_pruebas.withColumn(f"prueba_{p}", when(array_contains(col("pruebas_realizadas"), p), 1).otherwise(0))

# Eliminar la columna de lista de pruebas
df_pruebas = df_pruebas.drop("pruebas_realizadas")

In [ ]:
df_pruebas.show(10, truncate=False)

In [ ]:
# Agrupar por `id_ingreso` y obtener una lista de dominios de diagnóstico únicos
df_diagnosticos = df_dataset_filtrado.groupBy("id_ingreso").agg(
    collect_set("dominio_icd").alias("dominios")
)

df_diagnosticos.show(truncate=False)

In [ ]:
# Seleccionar id_ingreso y edad asegurando unicidad por ingreso
df_categorias = df_dataset_filtrado.select("id_ingreso", "edad").distinct()

# Crear la columna 'edad_categoria' con rangos
df_categorias = df_categorias.withColumn(
    "edad_categoria",
    when((df_categorias["edad"] >= 18) & (df_categorias["edad"] <= 39), "Joven")
    .when((df_categorias["edad"] >= 40) & (df_categorias["edad"] <= 64), "Adulto joven")
    .when((df_categorias["edad"] >= 65) & (df_categorias["edad"] <= 79), "Adulto mayor")
    .otherwise("Anciano")  # Para 80+ años
)

df_categorias.show(truncate=False)

In [ ]:
# Seleccionar las variables adicionales asegurando unicidad por `id_ingreso`
df_extra = df_dataset_filtrado.select(
    "id_ingreso", "sexo", "estado_civil", "tipo_seguro", "grupo_poblacional", "muerte_durante_ingreso"
).distinct()

df_extra.show()

In [ ]:
print(f"Numero de filas en df_diagnosticos: {df_diagnosticos.count()}")
print(f"Numero de filas en df_categorias: {df_categorias.count()}")
print(f"Numero de filas en df_extra: {df_extra.count()}")

In [ ]:
print(f"Numero de filas en df_dataset: {df_dataset.count()}")
print(f"Numero de ingresos en df_dataset: {df_dataset.select('id_ingreso').distinct().count()}")

In [ ]:
# Unir las tablas en un único dataset
df_arbol = df_pruebas \
    .join(df_diagnosticos, on="id_ingreso", how="inner") \
    .join(df_categorias, on="id_ingreso", how="inner") \
    .join(df_extra, on="id_ingreso", how="inner")


In [ ]:
df_arbol.persist()

In [ ]:
df_arbol.take(1)
df_arbol.show(truncate=False)

In [ ]:
df_arbol.count()

In [ ]:
output_path_parquet = f"data/resultados/dataset_arbol.parquet"

# Guardar en formato Parquet
df_arbol.write.mode("overwrite").parquet(output_path_parquet)

print(f"Dataset guardado en formato Parquet en: {output_path_parquet}")


Tras filtrar el dataset para conservar únicamente los diagnósticos presentes en las reglas de asociación, se realizan una serie de transformaciones para estructurar la información de manera óptima para el modelo de clasificación basado en árboles de decisión. 
- Primero, agregamos las pruebas de laboratorio (id_prueba) por paciente (id_ingreso), convirtiéndolas en una lista de pruebas realizadas. 
- Posteriormente, aplicamos One-Hot Encoding para transformar estas pruebas en variables dicotómicas, de modo que cada prueba se representara con una columna binaria (1 si la prueba fue realizada, 0 en caso contrario), lo que facilita la interpretación y uso en modelos supervisados.
- Además, agrupamos los diagnósticos por paciente en una lista (dominios), lo que nos permite capturar la presencia de múltiples condiciones médicas sin perder granularidad.
- Se incorporó la variable edad y se categorizó en cuatro grupos clínicamente relevantes: Joven (18-39 años), Adulto Joven (40-64 años), Adulto Mayor (65-79 años) y Anciano (80+ años). 
- Finalmente, integró la información sociodemográfica relevante (sexo, estado_civil, tipo_seguro, grupo_poblacional, muerte_durante_ingreso) asegurando unicidad por paciente. Todas estas tablas intermedias fueron combinadas en un único dataset (df_arbol) para su uso en el entrenamiento del modelo de decisión.